In [6]:
import torch
import os
import sys
sys.path.append('/home/ebutz/ESL2024/code/utils' )
import optuna
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.sampler import NegativeSampling
import torch_geometric.transforms as T
import pickle
import os
import sys
import torch
import torch.optim as optim
import torch_geometric
from torch_geometric.nn import ComplEx
from torch_geometric.data import Data
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from nxontology.imports import from_file
import wandb
from play_with_complex import *
from data_utils import *
from train_utils import *
from model_utils import *
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from torchmetrics.retrieval import RetrievalMRR, RetrievalHitRate
import play_with_complex as pwc
# Wandb
xp_name = 'First try GNN'
run_name = 'Baseline'

# Iric
iric_path = '/home/ebutz/ESL2024/data/full_iric/iric.csv'
mapped_iric_path  = '/home/ebutz/ESL2024/data/full_iric/altailed_mapped_iric.pickle'
altails_dict_path = '/home/ebutz/ESL2024/data/full_iric/altail_iric_DICT.pickle'
check_dicts = True

ontology_path = '/home/ebutz/ESL2024/data/go-basic.json.gz'

# Set device
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(f"Device: '{device}'")


run = wandb.init(project=xp_name, name = run_name)
config = wandb.config
config.val_ratio=0.1
config.homogeneous=False
config.scorelist_size=1000
config.split_ratio=0.8
config.val_ratio=0.1
config.test_ratio=0.1
config.num_neighbors=[70,55,13,89,85]
config.batch_size=1024
config.train_neg_sampling_ratio=224
config.epochs=18
config.disjoint_train_ratio=0.6
config.lr=0.0015308253347932983
config.stopper_metric= 'mrr'
config.stopper_direction="maximize"
config.stopper_patience=5
config.stopper_frequency=1
config.stopper_relative_delta=0.05
config.gamma=1.3
config.alpha=0.42680473078813763
config.gnn_layer='ResGatedGraphConv'
config.dropout=0.1
config.norm='DiffGroupNorm'
config.aggregation = 'min'
config.hidden_channels=115
config.num_layers=3
config.attention_heads=4
config.homogeneous = False
config.labels = {'head' : 'genes', 'relation' : 'gene_ontology', 'tail' : 'go'}

print(type(wandb.config))
print(config.homogeneous)
print(config.val_ratio)

data = load_iric_data(iric_path, featureless=False)
data  = T.ToUndirected(merge=True)(data) # Convert the graph to an undirected graph. Creates reverse edges for each edge.
data = T.RemoveDuplicatedEdges()(data) # Remove duplicated edges
print(data)
assert data.validate()

train_data, val_data, test_data = split_data(data, config)
train_loader, val_loader, test_loader = build_dataloaders(train_data, val_data, test_data, config)

batchy = next(iter(train_loader))


# # Model to train :
# hidden_channels = 176
# batch_size      = 4096
# epochs          = 200
# eval_period     = 2
# lin_factor      = 1

# use_wandb  = True

# params_save_name = f"PARAMS_ComplEx_6_times_{hidden_channels}_HC_{epochs}_epochs_{batch_size}_BS_on_full_iric"
# model_parameters_path = "/home/ebutz/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/"+params_save_name

# # Ontology
# ontology_path = "/home/ebutz/ESL2024/data/go-basic.json.gz"

# # ------------- Cuda ------------- #

# print("\nCuda check...")

# device = 'cpu'

# # ------------- Loading datas ------------- #

# iric = load_iric_data('data/iric.csv', featureless=False)
# display(iric)

# print("\nLoading iric...")
# mapped_iric = pd.read_pickle(mapped_iric_path)
# print(mapped_iric.head())
# print('mapped_alt_tails type :', type(mapped_iric.iloc[0]['mapped_alt_tails']))

# GO_to_map = mapped_iric.set_index('object')['mapped_object'].to_dict()
# map_to_GO = {value: key for key, value in GO_to_map.items()}

# if check_dicts:
#     looks_ok: bool = True
#     for i in tqdm(range(len(list(mapped_iric['object']))), desc = "Checking GO to MAP dict"):
#         if GO_to_map[mapped_iric['object'][i]]!=mapped_iric['mapped_object'][i] :
#             looks_ok = False
#     print('GO - Mapping dicts looks ok :', looks_ok)

# with open(altails_dict_path, 'rb') as handle:
#     mapped_alt_tails = pickle.load(handle)
# print("Alternative tails dict (first key-value pair):", list(mapped_alt_tails.items())[0])

# # ------------- Loading ontology ------------- #

print("\nLoading ontology...")
nxo = from_file(ontology_path)
nxo.freeze()
pwc.nxo = nxo

df = pd.read_csv(iric_path, index_col=0, dtype=str)
df['source_node'] = df.index
df.columns = [x.lower() for x in df.columns]
go_map = get_nodelist(df, 'go')
go_to_idx = {node: i for i, node in enumerate(go_map)}
idx_to_go = {i: node for i, node in enumerate(go_map)}

# # ------------- Making global variables accessibles to pwc ------------- #

pwc.map_to_GO        = idx_to_go
pwc.mapped_alt_tails = None
pwc.device           = device

# # ------------- Making datasets ------------- #


# print("\nDataset looks valid :",hetero_iric.validate(raise_on_error=True))

# print('All done.')


# # Init model


gnn_layers = get_gnn_layers(config)
norm_layers = get_norm_layers(config, len(data.node_types))

model = Model(config, data, norm_layers, gnn_layers).to(device)


# ----------------- Loops ----------------- #
@timer_func
def evaluate(config, loader, model, criterion, compute_all_metrics=False, loader_type='validation', stopper_metric=False):
    """
    Evaluate the model on a given data loader.

    Parameters
    ----------
    config : object
        An object containing the configuration parameters for the model.    
    loader : DataLoader
        The data loader to evaluate the model on.
    model : Model
        The model to evaluate.
    criterion : callable
        The loss function to use for evaluation.
    compute_all_metrics : bool, optional
        Whether to compute all metrics or partially. Default is False.
    loader_type : str, optional
        The type of the loader ('validation' or 'test'). Default is 'validation'.
    stopper_metric : str or bool, optional
        The metric that dictates when to stop training. If False, only the loss is computed. Default is False.

    Returns
    -------
    tuple
        A tuple of the evaluation metric and the loss, depending on the value of `stopper_metric`.
        If compute_all_metrics is True, the function logs all metrics to W&B.
    """
    model.eval()
    num_neg_samples = loader.neg_sampling.amount
    with torch.no_grad():
        if stopper_metric or compute_all_metrics:
            ground_truths = torch.tensor([], device='cpu')
            preds = torch.tensor([], device='cpu')
            indexes = torch.tensor([], device='cpu')
            total_loss, total_examples = 0, 0
            index_end = loader.batch_size

        for sampled_data in loader:
            sampled_data = sampled_data.to(device)

            batch_size = len(sampled_data[config.labels['tail']].dst_pos_index)
            pred = model(sampled_data, config)
            pos_samples = torch.ones(batch_size, device=device)
            neg_samples = torch.zeros(num_neg_samples*batch_size, device=device)
            ground_truth = torch.cat((pos_samples, neg_samples))

            if stopper_metric or compute_all_metrics: # Store preds and truths for all batches, and compute indices to calc metrics
                index_pos = torch.arange(end=index_end, start=index_end-batch_size) # index for predictions with pos. ground_truth
                index_neg = torch.arange(end=index_end, start=index_end-batch_size).repeat_interleave(num_neg_samples)
                index = torch.cat((index_pos, index_neg))
                indexes = torch.cat((indexes, index.to('cpu')))
                preds = torch.cat((preds, pred.to('cpu')))
                ground_truths = torch.cat((ground_truths, ground_truth.to('cpu')))
                index_end += batch_size

            eval_loss = criterion(pred, ground_truth, gamma=config.gamma, alpha=config.alpha)

            # if not stopper_metric: # Just logging loss
            #     wandb.log({"running_val_loss": eval_loss})
            #     break
            # else:
            total_loss += float(eval_loss) * pred.numel() 
            total_examples += pred.numel()
            eval_loss = total_loss / total_examples

            # if stopper_metric and not compute_all_metrics:
            #     if index_end >= 2048: # Compute approx. intermediate metric on a few datapoints to speed up hyperopt process
            #         break 

        model.train()

        if stopper_metric and not compute_all_metrics: # Compute only the metric that dictates stopping
            indexes = indexes.long()

            # # Log heatmap between prediction and ground truth. Useful for visualization / debugging. overhead 0.2s per epoch for a single sample.
            # heatmap = heatmaps(preds, ground_truths, indexes) 
            # wandb.log({f"heatmap": wandb.Image(heatmap)})
            # heatmap.close() # Free up memory

            match stopper_metric:
                case "val_loss":
                    eval_loss = total_loss / total_examples
                    return eval_loss
                
                case "mrr":
                    mrr = RetrievalMRR().to(device)
                    return mrr(preds, ground_truths, indexes=indexes), eval_loss
                
                case "hit_at_10":
                    hit_at_10 = RetrievalHitRate(top_k=10).to(device)
                    return hit_at_10(preds, ground_truths, indexes=indexes), eval_loss
                
                case "hit_at_5":
                    hit_at_5 = RetrievalHitRate(top_k=5).to(device)
                    return hit_at_5(preds, ground_truths, indexes=indexes), eval_loss
                
                case "hit_at_3":
                    hit_at_3 = RetrievalHitRate(top_k=3).to(device)
                    return hit_at_3(preds, ground_truths, indexes=indexes), eval_loss
                
                case "hit_at_1":
                    hit_at_1 = RetrievalHitRate(top_k=1).to(device)
                    return hit_at_1(preds, ground_truths, indexes=indexes), eval_loss
                
                case _:
                    raise ValueError(f"Unrecognized stopper metric: '{stopper_metric}'")
                
        if compute_all_metrics: # Compute all metrics at the end of training
            indexes = indexes.long()
            mrr = RetrievalMRR().to(device)
            hit_at_10 = RetrievalHitRate(top_k=10).to(device)
            hit_at_5 = RetrievalHitRate(top_k=5).to(device)
            hit_at_3 = RetrievalHitRate(top_k=3).to(device)
            hit_at_1 = RetrievalHitRate(top_k=1).to(device)
            
            mrr = mrr(preds, ground_truths, indexes=indexes)
            hit_at_10 = hit_at_10(preds, ground_truths, indexes=indexes)
            hit_at_5 = hit_at_5(preds, ground_truths, indexes=indexes)
            hit_at_3 = hit_at_3(preds, ground_truths, indexes=indexes)
            hit_at_1 = hit_at_1(preds, ground_truths, indexes=indexes)

            wandb.log({
                f"{loader_type}MRR": mrr, f"{loader_type}hit_at_10": hit_at_10, f"{loader_type}hit_at_5": hit_at_5, f"{loader_type}hit_at_3": hit_at_3, f"{loader_type}hit_at_1": hit_at_1
            })



model = Model(config, data, norm_layers, gnn_layers).to(device)
# print(model)

criterion = sigmoid_focal_loss
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
# optimizer = Lion(model.parameters(), lr=config.lr, weight_decay=1e-2)
early_stopper = EarlyStopper(frequency=config.stopper_frequency, patience=config.stopper_patience, direction=config.stopper_direction, relative_delta=config.stopper_relative_delta)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

@timer_func
def train(config, train_loader, val_loader, model, criterion, optimizer, early_stopper):
    for epoch in range(config.epochs):
        total_loss = total_examples = 0

        for sampled_data in tqdm(train_loader, desc="Training"):

            sampled_data = sampled_data.to(device)
            pred = model(sampled_data, config)
            # print(sampled_data)

            true_tails = sampled_data['dst_pos_index']

            pos_samples  = torch.ones(len(sampled_data[config.labels['tail']].dst_pos_index), device=device)
            neg_samples  = torch.zeros(len(sampled_data[config.labels['tail']].dst_neg_index.view(-1)), device=device) # As many zeroes as there are negative samples * batch_size
            ground_truth = torch.cat((pos_samples, neg_samples))
            # Add gaussian noise to pos_samples to simulate noisy labels. Added noise must be negative and not exceed 1.
            # ground_truth += torch.normal(mean=0, std=1, size=(len(ground_truth),), device=device)
            # apply sigmoid
            ground_truth = torch.sigmoid(ground_truth)
            loss = criterion(pred, ground_truth, gamma=config.gamma, alpha=config.alpha)
            wandb.log({"loss": loss})
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += float(loss) * pred.numel()
            total_examples += pred.numel()
        train_loss = total_loss / total_examples
        print(f"Epoch: {epoch:03d}, Avg. Loss: {train_loss:.10f}")
        # scheduler.step()

        # Compute metrics and check for early stopping
        score, val_loss = evaluate(config, val_loader, model, criterion, stopper_metric=config.stopper_metric)
        wandb.log({"avg_loss": train_loss, "val_loss": val_loss, f"{config.stopper_metric}": score})

        # early_stopper(score)
        # if early_stopper.early_stop:
        #     print("Early stopping triggered at epoch", epoch)
        #     break

    print("Training done.")

train(config, train_loader, val_loader, model, criterion, optimizer, early_stopper)
evaluate(config,  val_loader, model, criterion, compute_all_metrics = True, loader_type ='validation', stopper_metric = False)
evaluate(config, test_loader, model, criterion, compute_all_metrics = True, loader_type ='test'      , stopper_metric = False)

Device: 'cpu'


<class 'wandb.sdk.wandb_config.Config'>
False
0.1


/home/ebutz/ESL2024/code/utils/train_utils.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_links['source_node'] = df_links.index


HeteroData(
  genes={ x=[72335, 29] },
  go={ x=[42979, 3] },
  po={ x=[1662, 2] },
  traito={ x=[1636, 1] },
  prosite_profiles={ x=[627, 1] },
  prosite_patterns={ x=[652, 1] },
  superfamily={ x=[957, 1] },
  panther={ x=[5971, 1] },
  prints={ x=[420, 1] },
  (genes, interacts_with, genes)={ edge_index=[2, 1120849] },
  (genes, gene_ontology, go)={ edge_index=[2, 169248] },
  (genes, trait_ontology, traito)={ edge_index=[2, 15080] },
  (genes, plant_ontology, po)={ edge_index=[2, 6125] },
  (genes, profile, prosite_profiles)={ edge_index=[2, 12239] },
  (genes, pattern, prosite_patterns)={ edge_index=[2, 6677] },
  (genes, family, superfamily)={ edge_index=[2, 20742] },
  (genes, panther_id, panther)={ edge_index=[2, 25319] },
  (genes, prints_id, prints)={ edge_index=[2, 4365] },
  (go, is_a, go)={ edge_index=[2, 136436] },
  (po, is_a, po)={ edge_index=[2, 3548] },
  (traito, is_a, traito)={ edge_index=[2, 3770] },
  (go, rev_gene_ontology, genes)={ edge_index=[2, 169248] },
  (t

Training:   4%|▍         | 3/80 [00:21<09:12,  7.18s/it]


KeyboardInterrupt: 

In [8]:
print(batchy)

HeteroData(
  genes={
    x=[24914, 29],
    n_id=[24914],
    num_sampled_nodes=[6],
    src_index=[1024],
  },
  go={
    x=[42972, 3],
    n_id=[42972],
    num_sampled_nodes=[6],
    dst_pos_index=[1024],
    dst_neg_index=[1024, 224],
  },
  po={
    x=[1496, 2],
    n_id=[1496],
    num_sampled_nodes=[6],
  },
  traito={
    x=[1544, 1],
    n_id=[1544],
    num_sampled_nodes=[6],
  },
  prosite_profiles={
    x=[622, 1],
    n_id=[622],
    num_sampled_nodes=[6],
  },
  prosite_patterns={
    x=[652, 1],
    n_id=[652],
    num_sampled_nodes=[6],
  },
  superfamily={
    x=[950, 1],
    n_id=[950],
    num_sampled_nodes=[6],
  },
  panther={
    x=[5509, 1],
    n_id=[5509],
    num_sampled_nodes=[6],
  },
  prints={
    x=[419, 1],
    n_id=[419],
    num_sampled_nodes=[6],
  },
  dst_pos_index={},
  (genes, interacts_with, genes)={
    edge_index=[2, 488927],
    e_id=[488927],
    num_sampled_edges=[5],
  },
  (genes, gene_ontology, go)={
    edge_index=[2, 32512],
    edge_l

In [3]:
batchy['go']

NameError: name 'batchy' is not defined

torch.Size([1024])
tensor([39535, 10996, 39535,  ..., 26210, 13972, 13283])


In [48]:
print('Trues :')
true_tails = batchy['go']['dst_pos_index']
print(true_tails.size())
print(true_tails)
print('\nFalses :')
false_tails = batchy['go']['dst_neg_index']
print(false_tails.size())
print(false_tails)

print("\n-1-viewed falses :")
viewed_falses = batchy[config.labels['tail']].dst_neg_index.view(-1)
print(viewed_falses)
print(viewed_falses.size())

print("\nElonged_trues :")
elonged_trues = torch.repeat_interleave(true_tails,int(viewed_falses.size()[0]/true_tails.size()[0]))
print(elonged_trues.size())
print(elonged_trues)

Trues :
torch.Size([1024])
tensor([39535, 10996, 39535,  ..., 26210, 13972, 13283])

Falses :
torch.Size([1024, 224])
tensor([[ 2125, 24944,  9479,  ..., 29261,  6555, 32678],
        [39537, 33078, 21850,  ..., 14904, 41538, 24025],
        [27808, 19883, 14555,  ...,  4757, 25788, 16763],
        ...,
        [18989, 21549, 25492,  ..., 31450,  8316,   411],
        [35346, 28033,  8906,  ..., 41842, 12120, 11520],
        [29584, 38174, 38918,  ..., 31595, 20244, 33470]])

-1-viewed falses :
tensor([ 2125, 24944,  9479,  ..., 31595, 20244, 33470])
torch.Size([229376])

Elonged_trues :
torch.Size([229376])
tensor([39535, 39535, 39535,  ..., 13283, 13283, 13283])


In [55]:
linsims = pwc.lin_sims_for_batch(elonged_trues, viewed_falses)
print(linsims.size())
print(linsims)

tensor(10996)
tensor(40661)
torch.Size([229376])
tensor([0.0000, 0.0611, 0.0000,  ..., 0.0000, 0.0000, 0.2084])


In [57]:
pos_samples  = torch.ones(len(batchy[config.labels['tail']].dst_pos_index), device=device)
neg_samples  = torch.zeros(len(batchy[config.labels['tail']].dst_neg_index.view(-1)), device=device) # As many zeroes as there are negative samples * batch_size
lin_neg_samples = linsims
ground_truth = torch.cat((pos_samples, neg_samples))
lin_ground_truth = torch.cat((pos_samples, lin_neg_samples))

            
print('Pos Labels :')
print(pos_samples.size())
print(pos_samples)
print('\nNeg labels :')
print(neg_samples.size())
print(neg_samples)
print('\nLin Neg labels :')
print(lin_neg_samples.size())
print(lin_neg_samples)

Pos Labels :
torch.Size([1024])
tensor([1., 1., 1.,  ..., 1., 1., 1.])

Neg labels :
torch.Size([229376])
tensor([0., 0., 0.,  ..., 0., 0., 0.])

Lin Neg labels :
torch.Size([229376])
tensor([0.0000, 0.0611, 0.0000,  ..., 0.0000, 0.0000, 0.2084])


In [2]:
print(ground_truth.size())
print(lin_ground_truth.size())

NameError: name 'ground_truth' is not defined

In [29]:
pred = model(batchy, config)
print(pred.size())
print(pred)

torch.Size([230400])
tensor([ 0.5346,  0.4036,  0.5318,  ..., -0.0050, -0.0122,  0.0082],
       grad_fn=<ViewBackward0>)


In [61]:
loss = criterion(pred, ground_truth, gamma=config.gamma, alpha=config.alpha)
print(loss)
lin_loss = criterion(pred, lin_ground_truth, gamma=config.gamma, alpha=config.alpha)
print(lin_loss)

tensor(0.1625, grad_fn=<MeanBackward0>)
tensor(0.1590, grad_fn=<MeanBackward0>)


In [59]:
loss = criterion(pred, lin_ground_truth, gamma=config.gamma, alpha=config.alpha)
print(loss)

tensor(0.1590, grad_fn=<MeanBackward0>)


In [1]:
ground_truth += torch.normal(mean=0, std=1, size=(len(ground_truth),), device=device)
                # apply sigmoid
ground_truth = torch.sigmoid(ground_truth)
loss = criterion(pred, ground_truth, gamma=config.gamma, alpha=config.alpha)

NameError: name 'ground_truth' is not defined